In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Install SDG
 - git clone https://github.com/Red-Hat-AI-Innovation-Team/SDG-Research.git && cd SDG-Research
 - git checkout sdg_demo

In [ ]:
!pip install -r ../SDG-Research/requirements.txt
!pip install -e ../SDG-Research/.
!pip install rich datasets tabulate transformers
!pip install docling==1.20.0

In [1]:
# Third Party
from datasets import load_dataset
from openai import OpenAI
import click

# First Party
from instructlab.sdg.flow import Flow
from instructlab.sdg.pipeline import Pipeline
from instructlab.sdg.sdg import SDG
from instructlab.sdg.utils.docprocessor import DocProcessor
from utils.data import postprocess_and_save, pretty_print_dict

### Create Seed Data

In [22]:
!OMP_NUM_THREADS=32 mamba run -n docling python /workspace/home/lab/abhi/sdg_demo/SDG-Research/scripts/docparser.py --input-dir {data_dir} --output-dir {data_dir}

Not a conda environment: /workspace/home/lab/.conda/envs/docling

EnvironmentLocationNotFound: Not a conda environment: /workspace/home/lab/.conda/envs/docling



In [5]:
!python scripts/docparser.py --input-dir document_collection/RBC --output-dir document_collection/RBC

Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]
README.md: 100%|███████████████████████████| 3.49k/3.49k [00:00<00:00, 52.6MB/s]

model.pt:   0%|                                      | 0.00/201M [00:00<?, ?B/s]

tm_config.json: 100%|██████████████████████| 7.09k/7.09k [00:00<00:00, 81.9MB/s]


.gitattributes: 100%|██████████████████████| 1.71k/1.71k [00:00<00:00, 22.7MB/s]
Fetching 10 files:  10%|██▌                      | 1/10 [00:00<00:03,  2.37it/s]

otslp_all_standard_094_clean.check:   0%|            | 0.00/213M [00:00<?, ?B/s]


config.json: 100%|████████████████████████████| 41.0/41.0 [00:00<00:00, 732kB/s]



.gitignore: 100%|██████████████████████████| 5.18k/5.18k [00:00<00:00, 54.5MB/s]



model.pt:   0%|                                      | 0.00/169M [00:00<?, ?B/s]
model.pt:  10%|███▏                          | 21.0M/201M [00:00<00:01, 147MB/s]



otslp_all_fast.check:   0%|                          | 0.00/146M [00:00<?, ?B/s]

otslp_all_s

In [6]:
output_dir = f"sdg_demo_output/"
# This is where your PDFs are stored
data_dir = 'document_collection/RBC' 
# It also have your QNA yaml file
dp = DocProcessor(data_dir, user_config_path=f'{data_dir}/qna.yaml')
seed_data = dp.get_processed_dataset()
seed_data.to_json(f'{output_dir}/seed_data.jsonl', orient='records', lines=True)
pretty_print_dict(f'{output_dir}/seed_data.jsonl')


tokenizer_config.json:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

Map:   0%|          | 0/1350 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1404 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Generating train split: 0 examples [00:00, ? examples/s]

{
    "document_outline": "DB-SUSP-2ND-DAY-BAL Standard Operating Procedure (SOP)",
    "document_title": "RBC_SOP",
    "domain": "Standard Procedure Mannual",
    "icl_document": 
"-------------------------------------------------------------------------------------------------------|\n| All 
Transaction File  (ATF)   | Viewing manual entries (GL vouchers, credit and debit memos, and  Cheques) to do 
investigation                                                                                                      
|\n| Application Launch Pad        | Performing history inquiries in Unit Reference File Inquiry (URFIQ) - Unit  
Inquiry and Demand Deposit Account (DDA) - Deposit Services to obtain  details about transactions processed to 
personal and business client's  bank accounts. |\n| Chart of Account              | Retrieving the general 
information of the GL numbers.                                                                                     
|\n| Excel                         | Processing of $50 and under write-off items through the PeopleSoft bulk  
upload process. Also used for manual reconciliation of the Electronic Bill Payments GL.                            
|\n\n## DB-SUSP-2ND-DAY-BAL Standard Operating Procedure (SOP)\n\nVendor\n\nTarget Audience:\n\nVersion Number\n\n:
11.4\n\nVersion Date\n\n: February 26, 2024\n\nDate of Last Document Review : September 20, 2019\n\nNext Review 
Date\n\n: TBD\n| Image Retrieval  Integrated System (IRIS)   | Viewing manual entries (GL vouchers, credit and 
debit memos, and  transaction history) to do investigation.                                                        
|\n|---------------------------------------------|-----------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------|\n| OnDemand                                    | 
Viewing activity contained on the common reports required for  investigation.                                      
|\n| Outlook                                     | Getting email addresses and retrieving emails with attachments. 
Also used for email correspondence between the Service Partner and  Vendor or FSS for manual reconciliation of the 
Electronic Bill Payments  GL.                            |\n| PeopleSoft                                  | 
Creating GL account entries, searching for and viewing previously posted  journal entries.                         
|\n| RightFax FaxUtil                            | Sending faxes to Symcor centers.                                
|\n| Sales Platform                              | Viewing client profile and account details for the following:  
\u00b7  Current Account   Savings  \u00b7  Royal Credit Line(RCL)                                                  
|\n| Service Platform (SP)                       | Processing debits and/or credits entries to client's account and
also posting  current dated entries between client account and respective GL other than  Global Banking Service 
Centre (GBSC) and Payment and Trade (P And T).              |\n| Siebel                                      | 
Viewing branch financial and non-financial transactions posted via Service  Platform.                              
|\n| Transaction Lifecycle  Management (TLM)     | Monitoring, reconciling, and investigating GL accounts, and for 
completing  enrichment, balancing, and investigation for all types of transactions and  forwarding the 
communication to the respective business units and Service  Partners. |\n| Unit Reference File  Enquiry            
| Viewing the Sub transits and other transit details.     \",                                                      
| Matching DDA Transactions by Amount  Step 05                                                                     
|\n|------|----------------------------------------------

### Setup OpenAI Client for interacting with the model

In [ ]:
endpoint = f"https://xxx.com:443/v1"
openai_api_key = ""
openai_api_base = endpoint

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
teacher_model = client.models.list().data[0].id
print(teacher_model)

[14:42:38] INFO     HTTP Request: GET                                                               ]8;id=45149;file:///opt/app-root/lib64/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=297300;file:///opt/app-root/lib64/python3.11/site-packages/httpx/_client.py#1038\1038]8;;\
                    https://mixtral-8x7b-instruct-v0-1-maas-apicast-production.apps.prod.rhoai.rh-a                
                    iservices-bu.com/v1/models "HTTP/1.1 200 OK"                                                   

mistralai/Mixtral-8x7B-Instruct-v0.1


### Run SDG
- This will create knowledge flow from provided yaml file
- We will run this on small dataset for demo purposes
- For large scale generation, please use the python command provided in the next cell
- You can analyze the generated data to ensure the quality is similar to proivded QnA pairs

In [8]:
knowledge_agentic_pipeline = "src/instructlab/sdg/flows/generation/knowledge/synth_knowledge1.5.yaml"
flow_cfg = Flow(client).get_flow_from_file(knowledge_agentic_pipeline)
sdg = SDG(
    [Pipeline(flow_cfg)],
    num_workers=1,
    batch_size=1,
    save_freq=1000,
)

In [9]:
number_of_samples = 5
ds = load_dataset('json', data_files=f'{output_dir}/seed_data.jsonl', split='train')
ds = ds.shuffle(seed=42).select(range(number_of_samples))

In [10]:
# Checkpoint directory is used to save the intermediate datasets
generated_data = sdg.generate(ds, checkpoint_dir="Tmp")

[14:42:56] INFO     No existing checkpoints found in Tmp, generating from scratch                        ]8;id=871124;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py\sdg.py]8;;\:]8;id=249593;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py#107\107]8;;\

           INFO     Splitting the dataset into smaller batches                                           ]8;id=889511;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py\sdg.py]8;;\:]8;id=340942;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py#123\123]8;;\

100%|██████████| 5/5 [00:00<00:00, 119156.36it/s]


           INFO     Generating dataset with 5 splits, batch size 1, and 1 workers                        ]8;id=123555;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py\sdg.py]8;;\:]8;id=925267;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py#129\129]8;;\

           INFO     Processing split 0                                                                    ]8;id=458254;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py\sdg.py]8;;\:]8;id=419586;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py#75\75]8;;\

...

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Filter:   0%|          | 0/31 [00:00<?, ? examples/s]

Filter:   0%|          | 0/31 [00:00<?, ? examples/s]

           INFO     Finished future processing split 4                                                   ]8;id=789329;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py\sdg.py]8;;\:]8;id=574781;file:///opt/app-root/src/SDG-Research/src/instructlab/sdg/sdg.py#149\149]8;;\
                                                                                                                   
                                                                                                                   

100%|██████████| 5/5 [07:30<00:00, 90.12s/it]


### Run SDG through python command (For large scale generation)

```python
python /home/lab/sdg/scripts/generate.py --ds_path {output_dir}/seed_data.jsonl --bs 8 --num_workers 8 --save_path {output_dir}/gen.jsonl --flow SynthKnowledgeFlow1.5 --endpoint {teacher_endpoint_url} --checkpoint_dir {output_dir}/data_checkpoints --save_freq 2
```

### Save the generated data into training format

In [11]:
generated_data.to_json(f"{output_dir}/gen.jsonl", orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

741628

In [12]:
# Use the system prompt for RHELAI 1.4.1
system_prompt_rhelai_1_4_1 = (
    "I am a Red Hat® Instruct Model, an AI language model developed by Red Hat and IBM Research based on the granite-3.1-8b-base model. My primary role is to serve as a chat assistant."
)
precomputed_skills_path = "1.4.1/skills.jsonl"
# Download the RHELAI 1.4.1 data here: https://drive.google.com/file/d/1q8Rxcat5dZxXP-LqgPSCUsyttyAn6aLJ/view?usp=sharing
# Unzip the folder and put the path to skills.jsonl in precomputed_skills_path
postprocess_and_save(f"{output_dir}/gen.jsonl", dataset_save_path=f'{output_dir}', precomputed_skills_path=precomputed_skills_path, sys_prompt=system_prompt_rhelai_1_4_1)

Generating train split: 0 examples [00:00, ? examples/s]

num_proc must be <= 51. Reducing num_proc to 51 for dataset of size 51.


[15:20:44] WARNING  num_proc must be <= 51. Reducing num_proc to 51 for dataset of size 51.   ]8;id=180504;file:///opt/app-root/lib64/python3.11/site-packages/datasets/arrow_dataset.py\arrow_dataset.py]8;;\:]8;id=290605;file:///opt/app-root/lib64/python3.11/site-packages/datasets/arrow_dataset.py#3011\3011]8;;\

Filter (num_proc=51):   0%|          | 0/51 [00:00<?, ? examples/s]

num_proc must be <= 51. Reducing num_proc to 51 for dataset of size 51.


...

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=72):   0%|          | 0/346544 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/347 [00:00<?, ?ba/s]

[15:21:16] INFO     Saved datasets to sdg_demo_output/                                                  ]8;id=127891;file:///opt/app-root/src/SDG-Research/utils/data.py\data.py]8;;\:]8;id=832153;file:///opt/app-root/src/SDG-Research/utils/data.py#124\124]8;;\